In [ ]:
# install necessary packages (works on Google Colab even if you see a versioning error)
root = "/tmp/XCP-explain"
! git clone https://github.com/CPMpy/XCP-explain.git {root}
! cd {root} && git checkout ecai24
print("Installing packages (this can take a minute on Google Colab)")
! pip install -r /tmp/XCP-explain/requirements.txt -qq  # -qq=very quiet

import sys
if root not in sys.path:
    sys.path.insert(0, root)
print("Ready to go! (you can ignore Google Colab's google*/tensor* versioning errors)")

In [ ]:
# load necessary packages
import networkx as nx
import cpmpy as cp
import cpmpy.tools.explain as cpx

# graph drawing functions
draw = lambda g,**kwargs : nx.draw(g, nx.spring_layout(g, seed=42), width=4, node_size=500, **kwargs)
cmap = ["black", "yellow", "cyan", "lightgreen", "blue", "red", "magenta", "orange", "purple", "brown"]

import re
def graph_highlight(graph, cons, dotted=False, **kwargs):
    edges = []
    for c in cons:
        n1, n2 = c.args
        if n1.name == "max": continue
        a = int(re.search("\[[0-9]*\]", str(n1)).group()[1:-1])
        b = int(re.search("\[[0-9]*\]", str(n2)).group()[1:-1])
        edges.append((a,b))
        
    colors = ["red" if (a,b) in edges else "black" for (a,b) in graph.edges()]
    linestyles = ["dotted" if c == "red" and dotted else "solid" for c in colors]
    return draw(graph, edge_color=colors, style=linestyles, **kwargs)

## Let's generate a graph

Play with the parameters to get a different one!

In [ ]:
G = nx.fast_gnp_random_graph(9, 0.5, seed=21)
draw(G)

## How many colors do you think it needs?

In [ ]:
max_colors = 3  # YOUR GUESS FOR YOUR GRAPH

def graph_color_k(G, max_colors):
    # number of nodes
    n = G.number_of_nodes()

    # decision variables, one for every node
    x = cp.intvar(1, max_colors, shape=n, name="x")

    # constraints: neighbors have different colors
    m = cp.Model(
        [x[i] != x[j] for i,j in G.edges()],
    )
    return m, x

m, nodes = graph_color_k(G, max_colors)
if m.solve():
    print(m.status())
    print(f"Yes! There is a coloring with {max(nodes.value())} <= {max_colors} colors:")
    draw(G, node_color=[cmap[v.value()] for v in nodes])
else:
    print("No solution found, let the expaining begin : )")

## Given too few colors, lets find a deductive explanation: what causes the UNSAT?

In [ ]:
m, nodes = graph_color_k(G, max_colors)  # You can change the colors here
assert not m.solve(), "Choose too few colors to continue"

conflict = cpx.mus(m.constraints)  # Minimal Unsatisfiable Subset
print(f"UNSAT with {max_colors} colors is caused by the following minimal constraint set:")
print(conflict)
graph_highlight(G, conflict)

## Run it multiple times and it might find different conflicts...

In [ ]:
for i in range(50):
    conflict2 = cpx.mus(m.constraints)  # Minimal Unsatisfiable Subset
    if conflict2 != conflict:
        break  # found different conflict

print(f"UNSAT with {max_colors} colors is caused by the following minimal constraint set:")
print(conflict2)
graph_highlight(G, conflict2)

You might be surprised by the size of the of the MUSes, but we assure you these are subset minimal!

## OK, lets find a counterfactual explanation: what has to change to make it SAT?

In [ ]:
m, nodes = graph_color_k(G, max_colors)  # You can change the colors here
assert not m.solve(), "Choose too few colors to continue"

correction = cpx.mcs(m.constraints)  # Minimal Correction Subset
print(f"UNSAT with {max_colors} colors can be resolved by removing the following minimal constraint set:")
print(correction)
# compute and visualise counter-factual solution
cp.Model([c for c in m.constraints if c not in correction]).solve()
graph_highlight(G, correction, node_color=[cmap[n.value()] for n in nodes], dotted=True)
print("\n(removed constraints are highlighted in dotted red)")

## There are typically also many MCS's...

In [ ]:
for i in range(50):
    correction2 = cpx.mcs(m.constraints)  # Minimal Correction Subset
    if correction2 != correction:
        break  # found different conflict

print(f"UNSAT with {max_colors} colors can be resolved by removing the following minimal constraint set:")
print(correction2)
# compute and visualise counter-factual solution
cp.Model([c for c in m.constraints if c not in correction2]).solve()
graph_highlight(G, correction2, node_color=[cmap[n.value()] for n in nodes], dotted=True)
print("\n(removed constraints are highlighted in dotted red)")

## Higher-level constraints

In the graph coloring example each constraint is one atomic inequalty. The same techniques also work on 'complex' constraints, that is, a high-level constraint that when translated to a solver corresponds to a *group* of constraints.

### Here is an example: the Photo alignment problem.

Imagine: You're freshly graduated and starting your first job as a CS teacher at a local highschool. It's early in the morning and the students are coming in dressed in their finest of clothes. It's that time of the year again: the class photo! You're in charge of lining up the students for the class photo. Many of the students want to stand next to their friends and refuse to go on the photo otherwise. As a new teacher, you want to make sure that everyone is happy.


In [ ]:
wish_yes = [("Tias", "Dimos"), ("Tias", "Ignace"), ("Dimos", "Stella"), ("Ignace", "Helene"), ("Helene", "Dimos"), ("Stella", "Tias"), ("Ignace", "Thomas"), ("Lucifer", "Dimon")]

wish_no = [("Tias", "Lucifer"), ("Dimos", "Dimon"), ("Stella", "Lucifer")]

people = sorted(set(n for pair in wish_yes+wish_no for n in pair))
dmap = {name: index for index,name in enumerate(people)}  # name -> index
print(people)

position = cp.intvar(0,len(people)-1, shape=len(people), name=people)

# HARD constraint: everybody a unique position
con_diff = cp.alldifferent(position)

# Yes wishes must have difference in position of 1
cons_yes = []
for (a, b) in wish_yes:
    cons_yes.append( cp.abs(position[dmap[a]] - position[dmap[b]]) == 1 )
# No wishes can not have difference in position of 1
cons_no = []
for (a, b) in wish_no:
    cons_no.append( cp.abs(position[dmap[a]] - position[dmap[b]]) > 1 )

m = cp.Model(con_diff, cons_yes, cons_no)
print(m)
m.solve()

## It's unsat... try to find a MUS/MCS...

In this case, we will set the 'con_diff' as a HARD constraint.

So the MUS/MCS will only consider the cons_yes/cons_no

In [ ]:
#conflict = cpx.mus(cons_yes+cons_no, hard=[con_diff])  # Minimal Unsatisfiable Subset
#print(conflict)

In [ ]:
correction = []
#correction = cpx.mcs(cons_yes+cons_no, hard=[con_diff])  # Minimal Correction Subset
#print(correction)

In [ ]:
m2 = cp.Model(con_diff, [c for c in cons_yes+cons_no if hash(c) not in [hash(c) for c in correction]])
#print(m2)
if not m2.solve():
    print("No solution, try removing some constraints")
else:
    # print in the right order
    sol = [(pos.value(), str(pos)) for pos in position]
    print(sorted(sol))